In [2]:
# going to try to create my own word2vec embedding using gensim --> doing it here to save disk space
# the goal will be to train my model on the reddit relationship advice data

# I think the vocab length is way too long and this is why the kernel crashes
# going to use online version instead (with access to GPU) for actually esssay generation

# get gensim common_text corpus and word2vec model
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.word2vec import Text8Corpus
import numpy as np
import tensorflow as tf
from keras.layers import Embedding
from keras.engine import Input
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import LSTM
import keras
import string, re
from gensim.models import KeyedVectors

/Users/chloeloughridge/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
Using TensorFlow backend.


In [1]:
!pwd

/Users/chloeloughridge/TextGeneration


In [5]:
# train the word2vec thing on this and create vectors for each word in vocab

# saves to file, so no need to run more than once, I don't think

# inspiration: https://radimrehurek.com/gensim/models/word2vec.html
path = get_tmpfile("word2vec.model") # what does this do and do I need it?

sentences = Text8Corpus("/Users/chloeloughridge/TextGeneration/text8")
#model.build_vocab(sentences)  # prepare the model vocabulary --> do I actually need this? i think not, but oh well
model = Word2Vec(sentences, size=100, window=5, min_count=1, workers=4) # what are workers?
model.save("word2vec.model") # what is the advantage to doing this?

# save the word vectors as well
#word_vectors = model.wv
#fname = get_tmpfile("vectors.kv")
#word_vectors.save("vectors.kv")
model.wv.save_word2vec_format('vecs.bin', binary=True)

print(model['debod'])

[-0.00831181  0.01454275  0.02026019  0.01337432  0.01310148 -0.00104308
 -0.01053315 -0.00069205 -0.00892194 -0.00239479 -0.00321906  0.01468688
  0.00322119  0.0162561   0.00862454 -0.01703459  0.02949848 -0.00027297
 -0.00957121 -0.01190339  0.02782683 -0.01767031 -0.00361    -0.03497136
 -0.01871653  0.00938218  0.00059245 -0.00973722  0.00768029  0.00641198
 -0.03143131 -0.01235678 -0.02953943  0.02052058  0.01350835  0.00573183
 -0.00476676  0.00057476  0.0216789  -0.00174073  0.0317423  -0.02654162
  0.01337576  0.00167023 -0.00509533  0.02656584 -0.01549291 -0.00972257
  0.02090118  0.0034294  -0.00635527  0.00307668  0.00917678  0.00351963
 -0.00066626  0.00879129  0.00384175 -0.02436887  0.009249    0.02594081
  0.00156572 -0.01582196  0.00256946 -0.02396973  0.00921539  0.01957928
 -0.00825786 -0.00619234  0.01120604 -0.00774733 -0.02988713 -0.01793073
 -0.0119923  -0.00801529 -0.01506704 -0.00156928  0.01703234 -0.01327349
 -0.04125722 -0.01320669  0.02513841  0.00181677  0

/Users/chloeloughridge/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [4]:
# figure out how to restore the word2vec model --> will make this whole process faster

# define key variables
MAX_SEQUENCE_LENGTH = 100 # consider reducing this significantly
vocab_len = len(model.wv.vocab)


In [5]:
# function for creating embedding layer inspired by Ng course
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    ### START CODE HERE ###
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros([vocab_len, emb_dim])
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word in word_to_index:
        #print(word.index)
        index = word_to_index[word].index
        emb_matrix[index] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = Embedding(input_dim=vocab_len, output_dim=emb_dim, trainable=False)
    ### END CODE HERE ###

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [6]:
# building the model

input_shape = (vocab_len,)
sequence_input = Input(shape=input_shape)
# set up the embedding layer
embedding_layer = pretrained_embedding_layer(model, model.wv.vocab)
# send the input through the embedding layer 
embeddings = embedding_layer(sequence_input) 
print(embeddings.shape)
# now for the rest of the model
X = LSTM(28)(embeddings)
#X = Flatten()(X) --> we actually don't need the flatten layer
X = Dense(vocab_len, activation='softmax')(X)

# instantiate model
LSTM_model = Model(inputs=sequence_input, outputs=X)


/Users/chloeloughridge/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/chloeloughridge/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


(?, 253854, 100)


In [7]:
# compile the model
LSTM_model.compile(loss='binary_crossentropy',
              optimizer='adam', # or is rmsprop better??
              metrics=['binary_accuracy'])

In [8]:
# define a function for loading the text data from a file

def get_txt(file_path, header=True, delim='\n'):
    with open(file_path, 'r', encoding='utf8', errors='ignore') as f:
        if header:
            f.readline()
        texts = [line.rstrip(delim) for line in f]
        
    return texts


In [ ]:
# create cutom function for turning each line of text into one-hot vectors
orig_txt = get_txt('./reddit_relationshipadvice_legaladvice_2000.txt')

# create a matrix to store all of the one-hot vectors
X_input_data = np.zeros([len(orig_txt), MAX_SEQUENCE_LENGTH, len(model.wv.vocab)])

# will need to iterate through all of the lines
#for i in range(X_input_data.shape[0]):
# just testing
for i in range(1500):
    # takes a line and removes punctuation and splits into words
    s = orig_txt[i]
    exclude = set(string.punctuation)
    clean_txt = ''.join(ch for ch in s if ch not in exclude)
    clean_split = clean_txt.split()

    # for each word in this clean line, find the index in the vocab and conver to 
    # a one-hot vector
    index = 0
    for w in clean_split:
        try: 
            word_oh = keras.utils.to_categorical(model.wv.vocab[w.lower()].index, len(model.wv.vocab))
            # add to the X_input_data matrix
            X_input_data[i][index] = word_oh
        except:
            continue
        
        index = index + 1
    print(i)

print(X_input_data)
# iterate through every character in each line
    # if you haven't yet encountered any form of punctuation
        # add the letter to a list containing 

#print(test_txt)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
# running this line apparently kills the kernel
np.save('./X_input_data', X_input_data)

In [ ]:
# train the model 

# the Y data should be the X_input data shifted by one, cause you are predicting the next word
X_train = X_input_data[:1300, :X_input_data.shape[1] - 1, :]
Y_train = X_input_data[:1300, 1:, :]

X_test = X_input_data[1300:1500, :X_input_data.shape[1] - 1, :]
Y_test = X_input_data[1300:1500, 1:, :]

# actually, this may need to be reversed --> start with the zero vector for X1 --> that 
# corresponds to Y1

print(X_train[0].shape)
print(Y_train[0].shape)

# for each line in the X_train set, send through model
#for i in range(3):
    #print(X_train[1].shape)
    #LSTM_model.fit(X_train[i], Y_train[i])
    
LSTM_model.fit(X_train[0], Y_train[0])

#LSTM_model.fit(X_train, Y_train, batch_size= 28, validation_data=(X_test, Y_test))

In [ ]:
# sample the model

# choose a random starting word and then see where it goes from there?
